In [1]:
import pandas as pd
import numpy as np
import cv2
import torch
import os

In [2]:
# Check if CUDA is available
if torch.cuda.is_available():
    device = torch.device("cuda")
    print(f"Using GPU: {torch.cuda.get_device_name(0)}")
else:
    device = torch.device("cpu")
    print("Using CPU")

Using GPU: Tesla T4


# Transforming to tensors

In [26]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import numpy as np
import pandas as pd
from torch.utils.data import Dataset
from torchvision import transforms
from PIL import Image

In [51]:
classes = {
    0: "Ё",
    1: "А",
    2: "Р",
    3: "Е",
    4: "Ч",
    5: "Л",
    6: "Ц",
    7: "С",
    8: "Й",
    9: "З",
    10: "Ь",
    11: "Я",
    12: "Б",
    13: "Щ",
    14: "У",
    15: "Ы",
    16: "Д",
    17: "Ф",
    18: "Т",
    19: "Ъ",
    20: "О",
    21: "М",
    22: "П",
    23: "Х",
    24: "Э",
    25: "Н",
    26: "В",
    27: "Ю",
    28: "Ш",
    29: "Ж",
    30: "Г",
    31: "К",
    32: "И",
    33: "козел",
    34: "гнездо",
    35: "вой",
    36: "креветка",
    37: "бегемот",
    38: "нашествие",
    39: "лиса",
    40: "динозавр",
    41: "перо",
    42: "крокодил",
    43: "летучая мышь",
    44: "дикое животное",
    45: "попугай",
    46: "ежик",
    47: "волк",
    48: "овца",
    49: "петух",
    50: "белка",
    51: "пингвин",
    52: "помесь",
    53: "осел",
    54: "дятел",
    55: "воротник",
    56: "аллигатор",
    57: "кошка",
    58: "рога",
    59: "лошадь",
    60: "жираф",
    61: "вол",
    62: "населять",
    63: "верблюд",
    64: "коза",
    65: "ползанье",
    66: "бабочка",
    67: "клюв",
    68: "собака",
    69: "кролик",
    70: "паук",
    71: "водоплавающее",
    72: "носорог",
    73: "мышь",
    74: "зверь",
    75: "плавник",
    76: "паучья сеть",
    77: "животное",
    78: "лев",
    79: "тигр",
    80: "миграция",
    81: "копыто",
    82: "зубы",
    83: "коготь",
    84: "рыба",
    85: "лось",
    86: "пчела",
    87: "птица",
    88: "комар",
    89: "змея",
    90: "гавканье",
    91: "бык",
    92: "зоопарк",
    93: "клевать",
    94: "акула",
    95: "рычать",
    96: "панда",
    97: "крыса",
    98: "олень",
    99: "делать выводы",
    100: "орел",
    101: "лебедь",
    102: "рычание",
    103: "медведь",
    104: "слон",
    105: "дельфин",
    106: "павлин",
    107: "вилять",
    108: "собачий домик",
    109: "домашнее животное",
    110: "курица",
    111: "медуза",
    112: "клетка",
    113: "свинья",
    114: "лягушка",
    115: "голова",
    116: "корм для домашних животных",
    117: "корова",
    118: "обезьяна",
    119: "соты",
    120: "крыло",
    121: "заяц",
    122: "прирученный",
    123: "поверхностный",
    124: "интеллектуальный",
    125: "застенчивый",
    126: "добрый",
    127: "истерик",
    128: "аниматор",
    129: "кротость",
    130: "адаптивный",
    131: "амбициозный",
    132: "проныра",
    133: "скучный",
    134: "гордый",
    135: "одаренный",
    136: "прагматичный",
    137: "испорченный",
    138: "нежный",
    139: "любопытный",
    140: "гений",
    141: "рассеянный",
    142: "непослушный",
    143: "наивный",
    144: "острый",
    145: "вежливый",
    146: "отличный",
    147: "смелость",
    148: "правша",
    149: "разумный",
    150: "ленивый",
    151: "детский",
    152: "ребяческий",
    153: "стойкость",
    154: "уверенный",
    155: "обычный",
    156: "миленький",
    157: "остроумие",
    158: "послушный",
    159: "невиновный",
    160: "опытный",
    161: "инициатива",
    162: "тщеславный",
    163: "доминантный",
    164: "наблюдательность",
    165: "сильный",
    166: "вредный; озорной",
    167: "неловкий; неуклюжий",
    168: "верный",
    169: "особенный",
    170: "мятежный",
    171: "жестокий",
    172: "зрелищный",
    173: "строгий",
    174: "злобный",
    175: "нерадивый",
    176: "логичный",
    177: "торжественный",
    178: "враждебный",
    179: "получающий удовольствие",
    180: "потрясающий",
    181: "доминирующий",
    182: "жадный",
    183: "угрюмый",
    184: "трус",
    185: "жёсткий",
    186: "подлинный",
    187: "милосердный",
    188: "модный",
    189: "осторожный",
    190: "неудобно",
    191: "недобрый",
    192: "красивый",
    193: "жадность",
    194: "высокомерный",
    195: "приличный",
    196: "искренний",
    197: "аккуратный",
    198: "лицемерный",
    199: "положительный",
    200: "грубый",
    201: "ответственный",
    202: "отвратительный",
    203: "странный",
    204: "глупый",
    205: "адаптивное поведение",
    206: "отрицательный",
    207: "оживленный",
    208: "адаптивность",
    209: "объективный",
    210: "агрессивный",
    211: "предусмотрительный",
    212: "недовольный",
    213: "мудрый",
    214: "причинять резкую боль",
    215: "талант",
    216: "мужественность",
    217: "неприятный",
    218: "смешной",
    219: "запасливый",
    220: "милый",
    221: "трусливо",
    222: "образованный",
    223: "веселый",
    224: "дружелюбный",
    225: "нахальный",
    226: "личные качества",
    227: "беспокоящий",
    228: "наглость",
    229: "зло",
    230: "продуманный",
    231: "активный",
    232: "брюзжать",
    233: "скромный",
    234: "оранжевый",
    235: "основной цвет",
    236: "бледный",
    237: "тёмный",
    238: "серебро",
    239: "нераскрашенный",
    240: "розовый",
    241: "цветовой оттенок",
    242: "прозрачный",
    243: "оттенок",
    244: "коричневый",
    245: "бежевый",
    246: "непрозрачность",
    247: "цвет",
    248: "красноватый",
    249: "фиолетовый",
    250: "желтый",
    251: "темнота",
    252: "красный",
    253: "золотой",
    254: "черный",
    255: "розовато-лиловый",
    256: "серый",
    257: "синий",
    258: "зелёный",
    259: "белый",
    260: "грустный",
    261: "подавленный",
    262: "ошеломить",
    263: "нервный",
    264: "напряжение",
    265: "преданный",
    266: "мурашки",
    267: "облегчение",
    268: "ожидать",
    269: "зависть",
    270: "жажда",
    271: "слабый",
    272: "отношение",
    273: "отчаяться",
    274: "печаль",
    275: "озабоченный",
    276: "страх",
    277: "испуганный",
    278: "напуганный",
    279: "разрыдаться",
    280: "злость",
    281: "отвращение",
    282: "сочувствие",
    283: "неудобство",
    284: "самопроверка",
    285: "быть потрясенным",
    286: "доверять",
    287: "скорбеть",
    288: "смущенный",
    289: "желать",
    290: "вскружить",
    291: "самоанализ",
    292: "желание",
    293: "импульс",
    294: "презрение",
    295: "разъяренный",
    296: "самовосприятие",
    297: "беспокоиться",
    298: "возрадоваться",
    299: "подозревать",
    300: "терпеть",
    301: "влюбиться",
    302: "восхищаться",
    303: "нравиться",
    304: "злой",
    305: "расстроенный",
    306: "расслабление",
    307: "жуткий",
    308: "не выносить",
    309: "спокойный",
    310: "гнев",
    311: "сарказм",
    312: "отважиться",
    313: "сожалеть",
    314: "ленность",
    315: "расслабиться",
    316: "интересный",
    317: "самоосознание",
    318: "весенняя",
    319: "ненавидеть",
    320: "доброта",
    321: "отчаянный",
    322: "сомнение",
    323: "страшный",
    324: "покаяние",
    325: "испытывать",
    326: "счастливый",
    327: "испуг",
    328: "предпочитать",
    329: "дистресс",
    330: "возбуждение",
    331: "несчастье",
    332: "довольный",
    333: "благодарность",
    334: "голод",
    335: "изношенный",
    336: "счастье",
    337: "радость",
    338: "страдать",
    339: "мотивировать",
    340: "вдохновлять",
    341: "непреклонный",
    342: "рассердиться",
    343: "жалость",
    344: "безумно",
    345: "безразличие",
    346: "доставлять удовольствие",
    347: "изможденный",
    348: "добропорядочность",
    349: "любовный",
    350: "позор",
    351: "забота",
    352: "горе",
    353: "влюбленный",
    354: "рыдать",
    355: "вздрагивать",
    356: "самоконтроль",
    357: "ошарашенный",
    358: "неверие",
    359: "одержимый",
    360: "вдохновение",
    361: "самообман",
    362: "наслаждайся",
    363: "возбужденный",
    364: "любить",
    365: "терпение",
    366: "обнимать",
    367: "горький",
    368: "презирать",
    369: "беспокойство",
    370: "расстройство",
    371: "надоело",
    372: "слабость",
    373: "покраснеть",
    374: "голодный",
    375: "разгневанный",
    376: "ревнивый",
    377: "раздражать",
    378: "смириться",
    379: "обманчивость",
    380: "самомнение",
    381: "жаждущий",
    382: "возбуждать",
    383: "надеяться",
    384: "беспокойный",
    385: "мотивация",
    386: "пренебрежение",
    387: "благоговение",
    388: "суматоха",
    389: "горевать",
    390: "неуважение",
    391: "сомневаться",
    392: "нравственность",
    393: "горюющий",
    394: "надежда",
    395: "одинокий",
    396: "довольство",
    397: "агрессия",
    398: "страдание",
    399: "благополучие",
    400: "любовь",
    401: "боль",
    402: "скучающий",
    403: "домогаться",
    404: "разочарованный",
    405: "дрожать",
    406: "так себе",
    407: "стыдно",
    408: "вина",
    409: "весёлый",
    410: "страсть",
    411: "радостный",
    412: "объятие",
    413: "сожаление",
    414: "обеспокоенный",
    415: "рвеность",
    416: "гордость",
    417: "внутреннее",
    418: "одновременность",
    419: "восхищение",
    420: "поднос",
    421: "кафе",
    422: "лить",
    423: "откупоривать",
    424: "есть",
    425: "закуска перед едой",
    426: "кормить",
    427: "окунать",
    428: "бутылка",
    429: "жевать",
    430: "вкусный",
    431: "жадно есть",
    432: "переваривать",
    433: "MakDonalds",
    434: "предложить тост",
    435: "аллергия на еду",
    436: "откусывание",
    437: "ланч",
    438: "помощник повара",
    439: "кубок",
    440: "плата за вход",
    441: "резервирование; бронирование",
    442: "аппетит",
    443: "кафетерий",
    444: "нежный, мягкий",
    445: "пир",
    446: "прием пищи",
    447: "обедать",
    448: "кусок; тонкими слоями",
    449: "пить",
    450: "кружка",
    451: "кусать",
    452: "вилка",
    453: "делать мелкий глоток",
    454: "ароматный",
    455: "банкет",
    456: "заварочный чайник",
    457: "глотать",
    458: "кувшин",
    459: "ресторан",
    460: "переваривание",
    461: "глотать жадно",
    462: "официант",
    463: "заказывать",
    464: "женский туалет",
    465: "повар",
    466: "мужской туалет",
    467: "накрывать на стол",
    468: "вкус",
    469: "место для пикника",
    470: "диета",
    471: "питьевая вода",
    472: "Прощание",
    473: "С днем рождения",
    474: "Пока",
    475: "Позвонить на сервис",
    476: "Привет!",
    477: "Добро пожаловать!",
    478: "слово",
    479: "пока (что)",
    480: "верить",
    481: "иначе",
    482: "искать",
    483: "чемодан",
    484: "слышать",
    485: "вред",
    486: "десять",
    487: "почва",
    488: "сто",
    489: "достаточно",
    490: "пустой",
    491: "охотно",
    492: "значить",
    493: "собирать",
    494: "такой",
    495: "между",
    496: "карандаш",
    497: "маленький",
    498: "готовый",
    499: "восемь",
    500: "сказать",
    501: "восток",
    502: "иностранный",
    503: "платить",
    504: "дочь",
    505: "неделя",
    506: "семнадцать",
    507: "не",
    508: "двенадцатый",
    509: "писать",
    510: "целый",
    511: "часы",
    512: "конечно",
    513: "звук",
    514: "использовать",
    515: "быстрый",
    516: "семья",
    517: "четверг",
    518: "мужчина",
    519: "потом",
    520: "длинный",
    521: "отец",
    522: "ручка",
    523: "оба",
    524: "там",
    525: "тоже",
    526: "часть",
    527: "десятый",
    528: "дом",
    529: "новости",
    530: "лежать",
    531: "еще",
    532: "медленный",
    533: "семь",
    534: "возможный",
    535: "женщина",
    536: "рот",
    537: "муж",
    538: "жёлтый",
    539: "среда",
    540: "хорошо",
    541: "налог",
    542: "смеяться",
    543: "магазин",
    544: "другой",
    545: "путешествовать",
    546: "причина",
    547: "дешёвый",
    548: "книга",
    549: "еда",
    550: "шесть",
    551: "направление",
    552: "идти",
    553: "комната",
    554: "вчера",
    555: "один",
    556: "срочный",
    557: "известный",
    558: "двенадцать",
    559: "низкий",
    560: "пятый",
    561: "везде",
    562: "мы",
    563: "двадцать",
    564: "ноль",
    565: "должен",
    566: "почему",
    567: "девочка",
    568: "мир",
    569: "буду",
    570: "пятьдесят",
    571: "смотреть",
    572: "месяц",
    573: "короткий",
    574: "необходимый",
    575: "с",
    576: "если",
    577: "полный",
    578: "важный",
    579: "без",
    580: "человек",
    581: "немного",
    582: "завтра",
    583: "большой",
    584: "всегда",
    585: "молодой",
    586: "вниз",
    587: "стоять",
    588: "сестра",
    589: "ночь",
    590: "билет",
    591: "плакать",
    592: "имя",
    593: "или",
    594: "свободный",
    595: "мальчик",
    596: "говорить",
    597: "утро",
    598: "лето",
    599: "погода",
    600: "путешествие",
    601: "обучать",
    602: "пять",
    603: "сумка",
    604: "ребёнок",
    605: "север",
    606: "час",
    607: "очень",
    608: "место",
    609: "все",
    610: "ужасный",
    611: "держать",
    612: "одежда",
    613: "точно",
    614: "опасный",
    615: "седьмой",
    616: "много",
    617: "второй",
    618: "различный",
    619: "тридцать",
    620: "трудный",
    621: "два",
    622: "лучший",
    623: "одиннадцатый",
    624: "девятый",
    625: "быстро",
    626: "люди",
    627: "чувствовать",
    628: "письмо",
    629: "мочь",
    630: "друг",
    631: "назад",
    632: "шестьдесят",
    633: "улица",
    634: "уже",
    635: "правый",
    636: "сердце",
    637: "дорога",
    638: "высокий",
    639: "слушать",
    640: "кроме",
    641: "цена",
    642: "восьмой",
    643: "народ",
    644: "твердый",
    645: "мне",
    646: "тебе",
    647: "мертвый",
    648: "твой",
    649: "сын",
    650: "девяносто",
    651: "хлеб",
    652: "четвертый",
    653: "сегодня",
    654: "брат",
    655: "наконец",
    656: "постепенно",
    657: "рука",
    658: "весна",
    659: "хороший",
    660: "тот",
    661: "шум",
    662: "вода",
    663: "следующий",
    664: "шестнадцать",
    665: "безопасный",
    666: "несколько",
    667: "тысяча",
    668: "нас",
    669: "только",
    670: "воздух",
    671: "одиннадцать",
    672: "восемьдесят",
    673: "последний",
    674: "картина",
    675: "газета",
    676: "сторона",
    677: "способ",
    678: "переводить",
    679: "ждать",
    680: "особенно",
    681: "довольно",
    682: "бумага",
    683: "вероятно",
    684: "шестой",
    685: "обычно",
    686: "пятница",
    687: "под",
    688: "тот же самый (одинаковый)",
    689: "посылать",
    690: "четырнадцать",
    691: "с тех пор как",
    692: "вещь",
    693: "понять",
    694: "девятнадцать",
    695: "читать",
    696: "невозможный",
    697: "юг",
    698: "делать",
    699: "перед",
    700: "возможно",
    701: "понедельник",
    702: "жизнь",
    703: "голос",
    704: "грязный",
    705: "опять",
    706: "чистый",
    707: "три",
    708: "вопрос",
    709: "ничего",
    710: "город",
    711: "ты",
    712: "часто",
    713: "против",
    714: "их",
    715: "вечер",
    716: "ошибка",
    717: "прибывать",
    718: "я",
    719: "это",
    720: "осень",
    721: "наш",
    722: "школа",
    723: "вторник",
    724: "здесь",
    725: "веселиться",
    726: "глаз",
    727: "ключ",
    728: "сзади",
    729: "старый",
    730: "первый",
    731: "дорогой",
    732: "солнце",
    733: "что",
    734: "когда",
    735: "мой",
    736: "плохой",
    737: "четыре",
    738: "язык",
    739: "левый",
    740: "третий",
    741: "лучше",
    742: "видеть",
    743: "скоро",
    744: "деньги",
    745: "жена",
    746: "сверху",
    747: "худший",
    748: "суббота",
    749: "никогда",
    750: "воскресенье",
    751: "год",
    752: "новый",
    753: "зима",
    754: "никто",
    755: "время",
    756: "работать",
    757: "восемнадцать",
    758: "иногда",
    759: "однажды",
    760: "кто?",
    761: "до",
    762: "пятнадцать",
    763: "после",
    764: "семьдесят",
    765: "минута",
    766: "Плохо",
    767: "Даже",
    768: "Забрать",
    769: "Лицо",
    770: "Память",
    771: "Лопата",
    772: "Смех",
    773: "Футбол",
    774: "Выручить",
    775: "Соль",
    776: "Чашка",
    777: "Подробно",
    778: "Корона",
    779: "Рубль",
    780: "Крест",
    781: "Цемент",
    782: "Выигрыш",
    783: "Золото",
    784: "Жарить",
    785: "Пылесос",
    786: "Почки",
    787: "Свисток",
    788: "Чужой",
    789: "Религия",
    790: "Сразу",
    791: "Обладать",
    792: "Борода",
    793: "Витамин",
    794: "Раб",
    795: "остановить",
    796: "из-за",
    797: "чудесный",
    798: "боюсь",
    799: "закончить",
    800: "им",
    801: "эти",
    802: "ваш",
    803: "который",
    804: "немедленно",
    805: "нога",
    806: "ранний",
    807: "следовало",
    808: "остановиться",
    809: "поздний",
    810: "этот",
    811: "значит",
    812: "верх",
    813: "одолжить",
    814: "получить",
    815: "проверить",
    816: "замужем",
    817: "нуждаться",
    818: "открыть",
    819: "подруга",
    820: "остаться",
    821: "ясный",
    822: "минимум",
    823: "тогда",
    824: "над",
    825: "ей",
    826: "близко",
    827: "сюрприз",
    828: "изучить",
    829: "предложить",
    830: "терять",
    831: "теперь",
    832: "прекрасный",
    833: "около",
    834: "что-нибудь",
    835: "взять",
    836: "нам",
    837: "дать",
    838: "положить",
    839: "оставить",
    840: "сколько",
    841: "вас",
    842: "жаркий",
    843: "уронить",
    844: "слишком",
    845: "вы",
    846: "позволить",
    847: "привезти",
    848: "петь",
    849: "вам",
    850: "наверху",
    851: "наружу",
    852: "ехать",
    853: "те",
    854: "от",
    855: "прийти",
    856: "закрыть",
    857: "её",
    858: "всё",
    859: "женат",
    860: "бы",
    861: "показать",
    862: "весь",
    863: "сорок",
    864: "восемьсот",
    865: "пять тысяч",
    866: "три тысячи",
    867: "пятьсот",
    868: "двести",
    869: "четыреста",
    870: "семьсот",
    871: "тринадцать",
    872: "девять",
    873: "девятьсот",
    874: "четыре тысячи",
    875: "две тысячи",
    876: "триста",
    877: "миллион",
    878: "время от 0 ночи до 12 дня",
    879: "вспомнить",
    880: "бояться",
    881: "мыть",
    882: "свет",
    883: "найти",
    884: "обеспокоен",
    885: "запад",
    886: "сердитый",
    887: "жить",
    888: "шутить",
    889: "думать",
    890: "великий",
    891: "сухой",
    892: "спросить",
    893: "спать",
    894: "сначала",
    895: "хотеть",
    896: "знать",
    897: "цвета",
    898: "продолжить",
    899: "холодный",
    900: "да",
    901: "мать",
    902: "доволен",
    903: "легкий",
    904: "привык",
    905: "быть",
    906: "удивлён",
    907: "внизу",
    908: "рад",
    909: "болит",
    910: "согласно",
    911: "купить",
    912: "играть",
    913: "изменить",
    914: "ответить",
    915: "начать",
    916: "сквозь",
    917: "больше",
    918: "упасть",
    919: "вверх",
    920: "приятный",
    921: "тёплый",
    922: "уверен",
    923: "лечь",
    924: "забыть",
    925: "меня",
    926: "принести",
    927: "тебя",
    928: "встретить",
    929: "тяжёлый",
    930: "обещать",
    931: "день",
    932: "кусок",
    933: "помочь",
    934: "продать",
    935: "иметь",
    936: "назначенное время",
    937: "октябрь",
    938: "редко",
    939: "постоянный",
    940: "задерживать",
    941: "канун",
    942: "затем; потом",
    943: "ожидающий",
    944: "запоздалый",
    945: "6 часов",
    946: "поздно",
    947: "недавний",
    948: "в спешке",
    949: "распределять",
    950: "продлевать",
    951: "расписание",
    952: "откладывать",
    953: "немедленный",
    954: "время по Гринвичу",
    955: "день и ночь",
    956: "между тем",
    957: "ноябрь",
    958: "ранее",
    959: "продолжительность",
    960: "ежедневный",
    961: "будущее",
    962: "днем",
    963: "по часам",
    964: "две недели",
    965: "начинать",
    966: "вечный",
    967: "затягивать",
    968: "выходные",
    969: "бывший",
    970: "наконец-то",
    971: "временный",
    972: "восход солнца",
    973: "время для измерений",
    974: "в настоящее время",
    975: "в этом году",
    976: "краткий",
    977: "декабрь",
    978: "календарь",
    979: "7:45",
    980: "еще нет",
    981: "рано",
    982: "август",
    983: "в прошлом году",
    984: "отложить",
    985: "2 часа",
    986: "вечность",
    987: "дважды",
    988: "занятый",
    989: "начало",
    990: "отложенный",
    991: "мгновение",
    992: "неожиданный",
    993: "предыдущий",
    994: "в восемь пятнадцать",
    995: "продление",
    996: "башенные часы",
    997: "в то время как",
    998: "время 11 часов",
    999: "навсегда",
    1000: "---",
}

In [54]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [61]:
index = next((key for key, value in classes.items() if value == 'рычать'), None)
index

95

In [62]:
from torch.nn import functional as F
class PaddedSignLanguageDataset(Dataset):
    def __init__(self, annotations, transform=None, max_length=None):
        """
        Corrected custom dataset for loading sign language video tensors with padding.

        Args:
        annotations (DataFrame): DataFrame containing the annotations.
        transform (callable, optional): Optional transform to be applied on a sample.
        max_length (int, optional): Maximum length of the video tensors. If not provided, it will be calculated.
        """
        self.annotations = annotations
        self.transform = transform
        self.max_length = max_length
        self.shape = (64, 3, 224, 224)

        if self.max_length is None:
            # Calculate the maximum length among all tensors
            self.max_length = max(len(torch.load(row['attachment_id'], map_location=torch.device('cpu'))) for _, row in annotations.iterrows())

    def __len__(self):
        return len(self.annotations)

    def __getitem__(self, idx):
        tensor_path = self.annotations.iloc[idx]['attachment_id']
        label = self.annotations.iloc[idx]['text']
        label_id = next((key for key, value in classes.items() if value == label), None)
        # Load the tensor
        tensor = torch.load(tensor_path, map_location=torch.device('cpu'))

        # Check if the tensor is empty or None
        if tensor is None or len(tensor) == 0:
            print(f"Empty tensor found at index {idx}.")
            return None, label_id


        # Pad the tensor to the maximum length
#         empty = torch.empty(self.shape)
#         print(tensor.shape)
        tensor = F.pad(tensor, (0, 0, 0, 0, 0, 0, (64 - tensor.shape[1]) // 2, (64 - tensor.shape[1] + 1) // 2), "constant", 0)
#         print(tensor.shape)
        padded_tensor = torch.zeros((self.max_length, *tensor[0].shape))
#         c = (torch.stack(list_of_tensors,dim=1)).squeeze(0)
        padded_tensor[:len(tensor)] = torch.tensor(tensor, device='cuda').squeeze(0)
#         padded_tensor = torch.stack(padded_tensor, dim=0)
                
        # Apply transform if any
        if self.transform:
            padded_tensor = self.transform(padded_tensor)

        return (padded_tensor, label_id)

In [ ]:
# Load your data
annot_train150 = pd.read_csv("data/processed_annotations_train.csv")

# Load the first tensor
first_tensor_path = annot_train150['attachment_id'].iloc[0]
first_tensor = torch.load(first_tensor_path)

In [ ]:
annot_val50 = pd.read_csv("data/processed_annotations_valid.csv")

In [63]:
train_dataset = PaddedSignLanguageDataset(annot_train150, max_length=64)
val_dataset = PaddedSignLanguageDataset(annot_val50, max_length=64)

In [64]:
num_classes = 10
train_data_loader = DataLoader(train_dataset, batch_size=1, shuffle=True)
val_data_loader = DataLoader(val_dataset, batch_size=1, shuffle=False)

In [65]:
print(train_dataset[0][0].shape)

torch.Size([64, 64, 3, 224, 224])


/tmp/ipykernel_32/1003234596.py:44: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  padded_tensor[:len(tensor)] = torch.tensor(tensor, device='cuda').squeeze(0)


# Models

In [66]:
model_path = "/kaggle/input/mvit16-4/mvit16-4.pt"
model = torch.jit.load(model_path)

In [67]:
model.eval()
window_size = 16 # from model name
threshold = 0.5
frame_interval = 1
mean = [123.675, 116.28, 103.53]
std = [58.395, 57.12, 57.375]

In [68]:
model

RecursiveScriptModule(
  original_name=Recognizer3D
  (data_preprocessor): RecursiveScriptModule(original_name=ActionDataPreprocessor)
  (backbone): RecursiveScriptModule(
    original_name=MViT
    (patch_embed): RecursiveScriptModule(
      original_name=PatchEmbed3D
      (projection): RecursiveScriptModule(original_name=Conv3d)
    )
    (blocks): RecursiveScriptModule(
      original_name=ModuleList
      (0): RecursiveScriptModule(
        original_name=MultiScaleBlock
        (norm1): RecursiveScriptModule(original_name=LayerNorm)
        (attn): RecursiveScriptModule(
          original_name=MultiScaleAttention
          (qkv): RecursiveScriptModule(original_name=Linear)
          (proj): RecursiveScriptModule(original_name=Linear)
          (pool_q): RecursiveScriptModule(original_name=Conv3d)
          (norm_q): RecursiveScriptModule(original_name=LayerNorm)
          (pool_k): RecursiveScriptModule(original_name=Conv3d)
          (norm_k): RecursiveScriptModule(original_name

In [ ]:
prediction_list = []
prediction_list.append("---")

frame_counter = 0
for tensor, label in train_data_loader:
    i = 0
    while True:
        if i >= 4:
            break
        tensor16 = tensor[i:i+window_size]
        tensor16 = torch.reshape(tensor16, (64, 3, 64, 224, 224))
        input_tensor = np.stack(tensor16[:window_size], axis=1)[None][None]
        input_tensor = torch.from_numpy(input_tensor)
        with torch.no_grad():
            print(input_tensor.shape)
            outputs = model(input_tensor)[0]
        gloss = str(classes[outputs.argmax().item()])
        if outputs.max() > threshold:
            if gloss != prediction_list[-1] and len(prediction_list):
                if gloss != "---":
                    prediction_list.append(gloss)
            tensor16.clear()
            i += window_size

    text = "  ".join(prediction_list)
    text_div = np.zeros((50, frame.shape[1], 3), dtype=np.uint8)
    cv2.putText(text_div, text, (10, 30), cv2.FONT_HERSHEY_COMPLEX, 0.7, (255, 255, 255), 2)

    frame = np.concatenate((frame, text_div), axis=0)
#     writer.write(frame)
# writer.release()
# cap.release()

/tmp/ipykernel_32/1003234596.py:44: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  padded_tensor[:len(tensor)] = torch.tensor(tensor, device='cuda').squeeze(0)


torch.Size([64, 64, 3, 224, 224])
torch.Size([1, 1, 3, 16, 64, 224, 224])
torch.Size([1, 1, 3, 16, 64, 224, 224])
torch.Size([1, 1, 3, 16, 64, 224, 224])
torch.Size([1, 1, 3, 16, 64, 224, 224])
torch.Size([1, 1, 3, 16, 64, 224, 224])
torch.Size([1, 1, 3, 16, 64, 224, 224])
torch.Size([1, 1, 3, 16, 64, 224, 224])
torch.Size([1, 1, 3, 16, 64, 224, 224])
torch.Size([1, 1, 3, 16, 64, 224, 224])
torch.Size([1, 1, 3, 16, 64, 224, 224])
torch.Size([1, 1, 3, 16, 64, 224, 224])
torch.Size([1, 1, 3, 16, 64, 224, 224])
torch.Size([1, 1, 3, 16, 64, 224, 224])
torch.Size([1, 1, 3, 16, 64, 224, 224])
torch.Size([1, 1, 3, 16, 64, 224, 224])
torch.Size([1, 1, 3, 16, 64, 224, 224])
torch.Size([1, 1, 3, 16, 64, 224, 224])
torch.Size([1, 1, 3, 16, 64, 224, 224])
torch.Size([1, 1, 3, 16, 64, 224, 224])
torch.Size([1, 1, 3, 16, 64, 224, 224])
torch.Size([1, 1, 3, 16, 64, 224, 224])
torch.Size([1, 1, 3, 16, 64, 224, 224])
torch.Size([1, 1, 3, 16, 64, 224, 224])
torch.Size([1, 1, 3, 16, 64, 224, 224])
torch.